In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [6]:
import joblib

In [5]:
final_rmse = 5000

In [2]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [3]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler

In [4]:
def build_lstm_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [13]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    global final_rmse
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    lstm_loss_history = loss_history.history['loss']
    
    lstm_predictions = model.predict(X_test)
    lstm_predictions = scaler.inverse_transform(lstm_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, lstm_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    if rmse <= final_rmse:
        final_rmse = rmse
        joblib.dump(model, "BNB_model_lstm.pkl")
        plt.plot(y_test, label='Actual')
        plt.plot(lstm_predictions, label='Predicted')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.savefig(f'Plot_actualvspredicted/BNB_actual_vs_predicted.png')
        plt.close()
    
        plt.figure(figsize=(10, 6))
        plt.plot(loss_history.history['loss'], label='Training Loss')
        plt.plot(loss_history.history['val_loss'], label='Validation Loss')
        plt.title('Epoch Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'loss_curve/BNB_loss_curve.png')
        plt.close()
    
    return model, rmse


In [14]:
symbol = 'BNB-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'

In [15]:
epochs_range = [100]
batch_sizes = [4,8]
sequence_lengths = [25,60]
units_range = [50, 100]
layers_range = [2, 3]

In [16]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [17]:
results = []

In [18]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_lstm_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})


Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2
Epoch 1/100
268/268 [==============================] - 15s 29ms/step - loss: 0.0071 - val_loss: 0.0013
Epoch 2/100
  4/268 [..............................] - ETA: 5s - loss: 0.0035

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 6s 22ms/step - loss: 0.0040 - val_loss: 0.0011
Epoch 3/100
268/268 [==============================] - 6s 23ms/step - loss: 0.0022 - val_loss: 3.5518e-04
Epoch 4/100
268/268 [==============================] - 6s 22ms/step - loss: 0.0020 - val_loss: 9.7024e-04
Epoch 5/100
268/268 [==============================] - 6s 22ms/step - loss: 0.0015 - val_loss: 0.0024
Epoch 6/100
268/268 [==============================] - 6s 23ms/step - loss: 0.0014 - val_loss: 2.5751e-04
Epoch 7/100
268/268 [==============================] - 6s 22ms/step - loss: 0.0013 - val_loss: 2.6496e-04
Epoch 8/100
268/268 [==============================] - 6s 23ms/step - loss: 0.0010 - val_loss: 1.7913e-04
Epoch 9/100
268/268 [==============================] - 6s 22ms/step - loss: 9.7400e-04 - val_loss: 7.2293e-04
Epoch 10/100
268/268 [==============================] - 6s 23ms/step - loss: 0.0010 - val_loss: 1.4686e-04
Epoch 11/100
268/268 [==============================] - 6s 22

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 8s 30ms/step - loss: 0.0043 - val_loss: 0.0039
Epoch 3/100
268/268 [==============================] - 8s 30ms/step - loss: 0.0038 - val_loss: 4.3085e-04
Epoch 4/100
268/268 [==============================] - 8s 31ms/step - loss: 0.0024 - val_loss: 3.9449e-04
Epoch 5/100
268/268 [==============================] - 8s 30ms/step - loss: 0.0023 - val_loss: 4.0208e-04
Epoch 6/100
268/268 [==============================] - 8s 31ms/step - loss: 0.0016 - val_loss: 2.8111e-04
Epoch 7/100
268/268 [==============================] - 8s 31ms/step - loss: 0.0012 - val_loss: 1.8186e-04
Epoch 8/100
268/268 [==============================] - 8s 30ms/step - loss: 0.0010 - val_loss: 0.0026
Epoch 9/100
268/268 [==============================] - 8s 30ms/step - loss: 0.0010 - val_loss: 1.6522e-04
Epoch 10/100
268/268 [==============================] - 8s 29ms/step - loss: 9.6947e-04 - val_loss: 0.0022
Epoch 11/100
268/268 [==============================] - 8s 29ms/s

...

134/134 [==============================] - 12s 93ms/step - loss: 0.0044 - val_loss: 5.1095e-04
Epoch 3/100
134/134 [==============================] - 12s 90ms/step - loss: 0.0038 - val_loss: 7.3633e-04
Epoch 4/100
134/134 [==============================] - 12s 91ms/step - loss: 0.0028 - val_loss: 3.9131e-04
Epoch 5/100
134/134 [==============================] - 12s 91ms/step - loss: 0.0026 - val_loss: 0.0012
Epoch 6/100
134/134 [==============================] - 12s 90ms/step - loss: 0.0022 - val_loss: 0.0022
Epoch 7/100
134/134 [==============================] - 12s 90ms/step - loss: 0.0019 - val_loss: 0.0014
Epoch 8/100
134/134 [==============================] - 12s 92ms/step - loss: 0.0018 - val_loss: 2.3802e-04
Epoch 9/100
134/134 [==============================] - 13s 94ms/step - loss: 0.0015 - val_loss: 2.2154e-04
Epoch 10/100
134/134 [==============================] - 14s 103ms/step - loss: 0.0018 - val_loss: 0.0011
Epoch 11/100
134/134 [==============================] - 14s 103

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 17s 128ms/step - loss: 0.0029 - val_loss: 5.3956e-04
Epoch 3/100
134/134 [==============================] - 17s 124ms/step - loss: 0.0024 - val_loss: 0.0012
Epoch 4/100
134/134 [==============================] - 16s 116ms/step - loss: 0.0019 - val_loss: 2.6766e-04
Epoch 5/100
134/134 [==============================] - 16s 122ms/step - loss: 0.0021 - val_loss: 0.0017
Epoch 6/100
134/134 [==============================] - 16s 118ms/step - loss: 0.0014 - val_loss: 7.2981e-04
Epoch 7/100
134/134 [==============================] - 16s 120ms/step - loss: 0.0013 - val_loss: 3.0530e-04
Epoch 8/100
134/134 [==============================] - 16s 119ms/step - loss: 0.0012 - val_loss: 1.7358e-04
Epoch 9/100
134/134 [==============================] - 15s 109ms/step - loss: 9.9309e-04 - val_loss: 1.5260e-04
Epoch 10/100
134/134 [==============================] - 11s 83ms/step - loss: 9.3914e-04 - val_loss: 7.1240e-04
Epoch 11/100
134/134 [==================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 16s 121ms/step - loss: 0.0044 - val_loss: 0.0011
Epoch 3/100
134/134 [==============================] - 18s 134ms/step - loss: 0.0027 - val_loss: 5.7337e-04
Epoch 4/100
134/134 [==============================] - 18s 135ms/step - loss: 0.0024 - val_loss: 2.9685e-04
Epoch 5/100
134/134 [==============================] - 18s 135ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 6/100
134/134 [==============================] - 17s 124ms/step - loss: 0.0020 - val_loss: 3.1535e-04
Epoch 7/100
134/134 [==============================] - 17s 125ms/step - loss: 0.0016 - val_loss: 1.8298e-04
Epoch 8/100
134/134 [==============================] - 17s 126ms/step - loss: 0.0012 - val_loss: 6.5310e-04
Epoch 9/100
134/134 [==============================] - 17s 123ms/step - loss: 0.0011 - val_loss: 3.3447e-04
Epoch 10/100
134/134 [==============================] - 20s 148ms/step - loss: 9.9000e-04 - val_loss: 1.6823e-04
Epoch 11/100
134/134 [=====================

In [19]:
for result in results:
    print(result)

{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 6.949736407763667}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 7.025592495879779}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 6.049033785884647}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 7.528476597601788}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 2, 'rmse': 6.22992612544076}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 50, 'layers': 3, 'rmse': 10.624284810150742}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 2, 'rmse': 6.364121099381788}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 60, 'units': 100, 'layers': 3, 'rmse': 6.983796500444765}
{'epochs': 100, 'batch_size': 8, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 6.956974097047279}
{'epochs': 100,

In [20]:
model = joblib.load("BNB_model_lstm.pkl")

In [34]:
df1 = yf.download("BNB-USD","2024-01-01","2024-01-31")

[*********************100%%**********************]  1 of 1 completed


In [35]:
scaler1 = MinMaxScaler(feature_range=(0,1))

In [36]:
df1 = df1.filter(['Close']).values

In [37]:
df1_scaled = scaler1.fit_transform(df1)

In [38]:
last_25_days = df1_scaled[-25:]

In [39]:
future_prices = []
for _ in range(30):
    X_future = np.array([last_25_days])
    lstm_predicted_price = model.predict(X_future)
    last_25_days_lstm = np.append(last_25_days, lstm_predicted_price, axis=0)[1:]
    future_prices.append(lstm_predicted_price[0, 0])
    last_25_days = last_25_days_lstm.copy()  # 


1/1 [==============================] - 0s 105ms/step


In [40]:
future_prices = np.array(future_prices)
future_prices = future_prices.reshape(1,-1)
future_prices = scaler1.inverse_transform(future_prices)

In [41]:
yf.download("BNB-USD", "2024-02-1", "2024-02-20")['Close']

[*********************100%%**********************]  1 of 1 completed


Date
2024-02-01    300.175903
2024-02-02    301.154968
2024-02-03    299.736725
2024-02-04    304.734161
2024-02-05    300.854034
2024-02-06    302.710144
2024-02-07    307.634583
2024-02-08    318.871887
2024-02-09    323.842285
2024-02-10    323.133881
2024-02-11    320.787537
2024-02-12    327.944611
2024-02-13    324.868225
2024-02-14    334.245972
2024-02-15    354.710815
2024-02-16    360.890381
2024-02-17    352.908112
2024-02-18    349.679962
2024-02-19    351.807098
Name: Close, dtype: float64

In [42]:
future_prices

array([[306.7083 , 305.83725, 305.4069 , 305.48727, 305.92413, 306.45868,
        306.86816, 307.05743, 307.0495 , 306.9252 , 306.76953, 306.6432 ,
        306.57288, 306.55484, 306.56732, 306.58487, 306.5893 , 306.5739 ,
        306.5417 , 306.5006 , 306.45905, 306.4226 , 306.39334, 306.37012,
        306.35037, 306.33145, 306.3117 , 306.29056, 306.2684 , 306.24612]],
      dtype=float32)